In [1]:
# Instalar dependencias (ejecutar solo una vez por sesión)
!pip install pandas pydaymet datetime matplotlib ipywidgets openpyxl > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display, FileLink, HTML

def coordenadas_validas(lat, lon):
    # Validar rango de coordenadas para Daymet
    return 14.0 <= lat <= 52.0 and -130.0 <= lon <= -52.5

def obtener_datos_y_anio_representativo(lat, lon, CN_base):
    # Descargar datos diarios de Daymet y calcular año representativo
    inicio = date(1980, 1, 1)
    fin = date(date.today().year - 1, 12, 31)
    fechas = (inicio.strftime("%Y-%m-%d"), fin.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), fechas, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Fecha', 'prcp (mm/day)':'Precipitación'})[['Fecha', 'Precipitación']]
    df['Precipitación'] = df['Precipitación'].interpolate(method='linear', limit=1)
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Fecha'])
    df['Año'] = df['Fecha'].dt.year
    df['Mes'] = df['Fecha'].dt.month

    # Calcular escurrimiento base usando el método SCS-CN
    S_base = 25400 / CN_base - 254
    def escurrimiento(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Escurrimiento_base'] = df['Precipitación'].apply(lambda P: escurrimiento(P, S_base))

    # Encontrar año representativo según mediana anual de escurrimiento base
    anual_base = df.groupby('Año')['Escurrimiento_base'].sum().reset_index(name='Escurrimiento_anual_base')
    mediana_base = anual_base['Escurrimiento_anual_base'].median()
    anual_base['diferencia_mediana'] = (anual_base['Escurrimiento_anual_base'] - mediana_base).abs()
    min_diff = anual_base['diferencia_mediana'].min()
    candidatos = anual_base[anual_base['diferencia_mediana'] == min_diff]
    anio_rep = candidatos['Año'].max()
    return int(anio_rep), int(df['Año'].min()), int(df['Año'].max()), df

def mostrar_dashboard(nombre, lat, lon, CN_base, CN_obra, anio, mes_ini, mes_fin, variables, output):
    # Limpiar salida previa
    output.clear_output()
    with output:
        # Validar coordenadas antes de consultar Daymet
        if not coordenadas_validas(lat, lon):
            print(f"\nCoordenadas inválidas para Daymet.\nLatitud válida: 14.0 a 52.0\nLongitud válida: -130.0 a -52.5")
            print(f"Latitud actual: {lat:.4f}, Longitud actual: {lon:.4f}")
            return

        # Obtener datos y año representativo
        anio_rep, anio_min, anio_max, df = obtener_datos_y_anio_representativo(lat, lon, CN_base)
        if anio < anio_min or anio > anio_max:
            print(f"Año {anio} no disponible. Elige entre {anio_min} y {anio_max}.")
            return

        # Calcular escurrimiento base y con obras
        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def escurrimiento(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Escurrimiento_base'] = df['Precipitación'].apply(lambda P: escurrimiento(P, S_base))
        df['Escurrimiento_obra'] = df['Precipitación'].apply(lambda P: escurrimiento(P, S_obra))
        df['Evento_base'] = df['Precipitación'] > 0.2 * S_base
        df['Evento_obra'] = df['Precipitación'] > 0.2 * S_obra

        # Filtrar año para métricas y gráficas
        df_anio = df[df['Año'] == anio]

        # Calcular métricas anuales
        esc_base_anual = df_anio['Escurrimiento_base'].sum()
        esc_obra_anual = df_anio['Escurrimiento_obra'].sum()
        diferencia_mm = esc_base_anual - esc_obra_anual
        diferencia_m3ha = diferencia_mm * 10
        eventos_base = df_anio['Evento_base'].sum()
        eventos_obra = df_anio['Evento_obra'].sum()
        pp_anual = df_anio['Precipitación'].sum()
        coef_base = esc_base_anual / pp_anual if pp_anual > 0 else 0
        coef_obra = esc_obra_anual / pp_anual if pp_anual > 0 else 0

        # Mostrar resumen de métricas anuales
        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {anio_rep}")
        print(f"Año seleccionado (completo): {int(anio)}")
        print(f"Escurrimiento base (año completo): {esc_base_anual:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {esc_obra_anual:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra)}")
        print(f"Coef. escurrimiento base: {coef_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_obra:.2%}")

        # Filtrar rango de meses para hidrograma mostrado
        df_mes = df_anio[(df_anio['Mes'] >= mes_ini) & (df_anio['Mes'] <= mes_fin)]

        # Graficar hidrograma del rango de meses seleccionado
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_mes['Fecha'], df_mes['Precipitación'], color='#0055B8', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_mes['Fecha'], df_mes['Escurrimiento_base'], color='#FF6469', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_mes['Fecha'], df_mes['Escurrimiento_obra'], color='#32CC68', label='Escurrimiento con obras (mm)', linewidth=1)
        plt.title(f'Hidrograma año representativo: {int(anio)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Graficar barras comparativas anuales
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [esc_base_anual, esc_obra_anual, diferencia_mm], color=['#FF6469','#80CE9F','#0055B8'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño representativo: {int(anio)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Preparar datos exportables: siempre exportar Fecha, Precipitación, Escurrimiento base y con obras
        global ultimo_df_export
        columnas_exportar = ['Fecha', 'Precipitación', 'Escurrimiento_base', 'Escurrimiento_obra']
        nombres_exportar = ['Fecha', 'Precipitación (mm)', 'Escurrimiento base (mm)', 'Escurrimiento con obras (mm)']
        ultimo_df_export = df_anio[columnas_exportar].sort_values('Fecha').copy()
        ultimo_df_export.columns = nombres_exportar

        # Mostrar enlace para descargar el archivo si existe
        if ultimo_df_export is not None and len(ultimo_df_export) > 0:
            nombre_archivo = f"{nombre}_{anio}.xlsx"
            ultimo_df_export.to_excel(nombre_archivo, index=False)
            print(f"\nArchivo Excel creado: {nombre_archivo}")
            display(FileLink(nombre_archivo))
            display(HTML(f'<a href="{nombre_archivo}" download>Haz clic aquí para descargar el archivo Excel</a>'))
        else:
            print("No hay datos para exportar. Cambia parámetros y vuelve a intentar.")

# Definir widgets interactivos
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

# Obtener año representativo y rango de años (coordenadas válidas por defecto)
anio_rep, anio_min, anio_max, df_init = obtener_datos_y_anio_representativo(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
anio = IntText(value=anio_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
output = Output()

ultimo_df_export = None  # Variable global para exportación

def obtener_variables_seleccionadas():
    # Obtener variables seleccionadas para mostrar en la gráfica
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    # Actualizar dashboard cuando cambia algún parámetro
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    mostrar_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        anio.value, mes_ini.value, mes_fin.value,
        obtener_variables_seleccionadas(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, anio, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def asignar_anio_rep(b):
    # Asignar año representativo como año seleccionado
    anio_rep, _, _, _ = obtener_datos_y_anio_representativo(lat.value, lon.value, CN_base.value)
    anio.value = anio_rep

btn_rep.on_click(asignar_anio_rep)

# Mostrar interfaz gráfica interactiva
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, anio, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    output
]))
actualizar_dashboard()